In [ ]:
import pandas as pd
import numpy as np

In [ ]:
ptsd_df = pd.read_csv('/content/detailed_lables.csv')

In [ ]:
ptsd = ptsd_df[['Participant', 'PTSD_label']]
ptsd_dict = dict(zip(ptsd['Participant'], ptsd['PTSD_label']))

In [ ]:
Depression = ptsd_df[['Participant', 'Depression_label']]
Depression_dict = dict(zip(Depression['Participant'], Depression['Depression_label']))

In [ ]:
dl_df = pd.read_csv('/content/detailed_lables.csv')

In [ ]:
bins = [0, 1, 3]
lab = [0, 1]
dl_df['Appetite'] = pd.cut(dl_df['PHQ8_5_Appetite'], bins=bins, labels=lab, right=True, include_lowest=True)
dl_df['Appetite'] = dl_df['Appetite'].astype(int)

In [ ]:
appetite = dl_df[['Participant', 'Appetite']]
appetite_dict = dict(zip(appetite['Participant'], appetite['Appetite']))

In [ ]:
dl_df = pd.read_csv('/content/detailed_lables.csv')

In [ ]:
dl_df['PHQ_Agency'] = 12 - (
    dl_df['PHQ8_1_NoInterest'] +
    dl_df['PHQ8_2_Depressed'] +
    dl_df['PHQ8_6_Failure'] +
    dl_df['PHQ8_7_Concentration']
) / 9

In [ ]:
dl_df['PCLC_Agency'] = (((dl_df['PCL-C_8_TroubleRemembering'] + dl_df['PCL-C_9_NoInterest'] + dl_df['PCL-C_10_FeelDistant'] +
                        dl_df['PCL-C_11_FeelNumb'] + dl_df['PCL-C_12_Future_Cut_Short'] +
                        dl_df['PCL-C_13_Sleep'] +
                        dl_df['PCL-C_14_Irritability']) / 35) + ((dl_df['PCL-C_15_Concentration'] +
                        dl_df['PCL-C_16_HyperAlert'] + dl_df['PCL-C_17_Jumpy'])/15))*0.5

In [ ]:
# Step 1: Take the average of the two agency scores
dl_df['Avg_Agency'] = (dl_df['PCLC_Agency'] + dl_df['PHQ_Agency']) / 2

# Step 2: Compute the median of the average
median_value = dl_df['Avg_Agency'].median()

# Step 3: Assign 0 if less than median, 1 if greater than or equal to median
dl_df['Agency_Split'] = (dl_df['Avg_Agency'] >= median_value).astype(int)

In [ ]:
agency = dl_df[['Participant', 'Agency_Split']]
agency_dict = dict(zip(agency['Participant'], agency['Agency_Split']))

In [ ]:
# Step 1: Define anxiety-related columns from PHQ-8 and PCL-C
anxiety_items = [
    'PHQ8_3_Sleep',
    'PHQ8_4_Tired',
    'PHQ8_7_Concentration',
    'PHQ8_8_Psychomotor',
    'PCL-C_13_Sleep',
    'PCL-C_14_Irritability',
    'PCL-C_15_Concentration',
    'PCL-C_16_HyperAlert',
    'PCL-C_17_Jumpy'
]

# Step 2: Compute the sum score across those columns
dl_df['Anxiety_Score'] = dl_df[anxiety_items].sum(axis=1)

# Step 3: Compute median of Anxiety_Score
median_anxiety = dl_df['Anxiety_Score'].median()

# Step 4: Assign binary anxiety label based on median split
dl_df['Anxiety_Label'] = (dl_df['Anxiety_Score'] >= median_anxiety).astype(int)


In [ ]:
anxiety = dl_df[['Participant', 'Anxiety_Label']]
anxiety_dict = dict(zip(anxiety['Participant'], anxiety['Anxiety_Label']))

In [ ]:
dl_df = pd.read_csv('/content/Sleep_Binary_labels.csv')
sleep = dl_df[['Participant', 'Sleep_Disorder']]
sleep_dict = dict(zip(sleep['Participant'], sleep['Sleep_Disorder']))

In [ ]:
ptsd_df = pd.DataFrame(list(ptsd_dict.items()), columns=['Participant_ID', 'PTSD_Label'])
depression_df = pd.DataFrame(list(Depression_dict.items()), columns=['Participant_ID', 'Depression_Label'])
appetite_df = pd.DataFrame(list(appetite_dict.items()), columns=['Participant_ID', 'Appetite_Label'])
agency_df = pd.DataFrame(list(agency_dict.items()), columns=['Participant_ID', 'Agency_Label'])
anxiety_df = pd.DataFrame(list(anxiety_dict.items()), columns=['Participant_ID', 'Anxiety_Label'])
sleep_df = pd.DataFrame(list(sleep_dict.items()), columns=['Participant_ID', 'Sleep_Label'])

# Step 2: Merge all DataFrames on 'Participant_ID'
merged_df = ptsd_df \
    .merge(depression_df, on='Participant_ID', how='outer') \
    .merge(appetite_df, on='Participant_ID', how='outer') \
    .merge(agency_df, on='Participant_ID', how='outer') \
    .merge(anxiety_df, on='Participant_ID', how='outer') \
    .merge(sleep_df, on='Participant_ID', how='outer')


# Step 3: Save to CSV
merged_df.to_csv('mental_health_labels.csv', index=False)